In [20]:
from src.dataset import DataSet
from src.dataset_parser import parse_data_to_csv
from src.baseline import dummy

In [27]:
raw_path = "./original_data"
parsed_path = "./parsed_data"
# parse_data_to_csv(raw_path, parsed_path) // DO NOT EXECUTE UNLESS PARSED DATA IS LOST

Parsed data to ./parsed_data CSV successfully.


In [28]:
ds = DataSet(parsed_path)

In [29]:
# print(ds.training, ds.training.dtypes)
dummy.fit(ds.training["review_text"], ds.training["polarity"])

DummyClassifier(strategy='most_frequent')

In [30]:
dummy.predict(ds.training["review_text"])

array(['positive', 'positive', 'positive', ..., 'positive', 'positive',
       'positive'], dtype='<U8')

In [26]:
dummy.score(ds.training["review_text"], ds.training["polarity"])

0.50375